In [4]:
#importing essential packages
!pip install sklearn
!pip install nltk
!pip install tensorflow
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from bs4 import BeautifulSoup as bf
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer 
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as k

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
data = pd.read_csv("IMDB Dataset.csv")

#cleaning the text
label_encoder = preprocessing.LabelEncoder()
porter = PorterStemmer()

for i in range(50000):
  data.loc[i][0] = bf(data.loc[i][0],'html.parser').get_text()

stop = stopwords.words("english")

for i in range(50000):
  data.loc[i][0] = [x for x in word_tokenize(data.loc[i][0]) if not x in stop]
  data.loc[i][0] = [x for x in data.loc[i][0] if str(x).isalpha()]
  data.loc[i][0] = [porter.stem(x) for x in data.loc[i][0]]
  data.loc[i][0] = " ".join(x for x in data.loc[i][0]) 

In [0]:
#preparing data for training
count = TfidfVectorizer(max_features=5000)
data.sentiment =  [x for x in label_encoder.fit_transform(data.sentiment)]

count.fit(data.review)
vector = count.transform(data.review)

from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(vector.toarray(),data.sentiment,test_size = 0.2)

In [15]:
#building the model 
from tqdm import tqdm

model = tf.keras.models.Sequential([
           k.Dense(250,activation = 'relu',input_dim = X_train.shape[1]),
           k.Dense(1,activation = 'sigmoid'),
])

model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics=['accuracy'])
for i in tqdm(range(2)):
  model.fit(X_train, Y_train, validation_split = 0.2, verbose = 2)
_,score = model.evaluate(X_test,Y_test)
score = str(score*100)+str('%')
print("The accuracy of the model is {}".format(score))

  0%|          | 0/2 [00:00<?, ?it/s]

1000/1000 - 3s - loss: 0.3258 - accuracy: 0.8610 - val_loss: 0.2831 - val_accuracy: 0.8811


 50%|█████     | 1/2 [00:04<00:04,  4.23s/it]

1000/1000 - 3s - loss: 0.2367 - accuracy: 0.9038 - val_loss: 0.2896 - val_accuracy: 0.8811


100%|██████████| 2/2 [00:08<00:00,  4.27s/it]


313/313 [==============================] - 1s 2ms/step - loss: 0.2911 - accuracy: 0.8775
The accuracy of the model is 87.74999976158142%
